In [1]:
from ecmwf.opendata import Client
import xarray as xr
import pandas as pd

from datetime import datetime
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))  # set vscode notebook path for module imports

from utils.tools import regions_from_xarray

In [2]:
steps = [i for i in range(0, 144, 3)] + [j for j in range(144, 361, 6)] #get all steps
time = 0 if datetime.now().hour < 18 else 12 #get 00z forecast if we are before 12z, else get 12z forecast
current_date = datetime.now().date() #select current date forecast
var = '2t' #forecast returned variables

In [ ]:
#Download forecast data
client = Client()
result = client.retrieve(
    date=current_date, #can be 0 for today, -1 yestarday... if 
    time=time,
    type="fc",  #forecast for HRES
    stream='oper',
    step=steps,
    param=var,
    target="current.grib2",
)

In [3]:
#Open the forecast dataset and format the datas
ds = xr.open_dataset('current.grib2')
forecast_date = ds.time.values #keep track of the forecast run date time
us = ds.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 
us = (us - 273.15) * 1.8 + 32 #Convert kelvin to °F
us.attrs['units'] = '°F'

us = us.swap_dims({"step": "valid_time"})

#We slice the dataset with valid_times to avoid non complete day between 00z run (complete days) and 12z run (non complete days)
us = us.sel(valid_time=slice(pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=1), #set the dataset at the start of the next day for our first window
                             pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=14, hours=23))) #and just before the last valid_time

#Now that we start the first day at 00z for 00z run AND 12z run, resample hourly to daily
us_daily = us.resample(valid_time="1D").mean()

us_daily_hdd = (65 - us_daily).clip(min=0) #compute HDD

Ignoring index file 'current.grib2.5b7b6.idx' older than GRIB file


In [4]:
pop = xr.open_dataarray('../utils/files/population_regridded_025deg.nc')

In [5]:
hdd_weighted = us_daily_hdd * pop #Weight the hdd by population for each point in the grid and each valid_time

In [6]:
horizons = [(0,2, 'Day 1-3'), (3,6, 'Day 4-7'), (7,13, 'Day 8-14')]
hdd_list = []
for horizon in horizons:
    #Create horizon date for slicing
    first_date = hdd_weighted.valid_time.min().values
    start_date = first_date + pd.Timedelta(days=horizon[0])
    last_date = first_date + pd.Timedelta(days=horizon[1])

    hdd_weighted_horizon = hdd_weighted.sel(valid_time=slice(start_date, last_date)).sum(dim='valid_time') #sumed HDD for every grid point in the horizon

    #Make US mean
    us_horizon_mean = hdd_weighted_horizon.mean(dim=['latitude', 'longitude']) #Mean every point in the US to have one mean weighted HDD for the horizon
    hdd_list.append({'forecast_run_time': forecast_date, 'region': 'US Mean', 'horizon_start': start_date, 'horizon_end': last_date,
                    'horizon_label': horizon[2], 'forecast_HDD': us_horizon_mean.t2m.values})
    
    #Make region means
    zone_means = regions_from_xarray(hdd_weighted_horizon)

    #For each zone in the horizon, make a new row of data
    for zone_name, zone_data in zone_means.items():
        hdd_list.append({'forecast_run_time': forecast_date, 'region': zone_name, 'horizon_start': start_date, 'horizon_end': last_date,
                         'horizon_label': horizon[2], 'forecast_HDD': zone_data.t2m.values})
        
#Make a dataframe of values
hdd_horizon_df = pd.DataFrame(hdd_list)

In [7]:
hdd_horizon_df.head()

,forecast_run_time,region,horizon_start,horizon_end,horizon_label,forecast_HDD
0,2026-01-29 12:00:00,US Mean,2026-01-30,2026-02-01,Day 1-3,1880703.6095141533
1,2026-01-29 12:00:00,Great Lakes,2026-01-30,2026-02-01,Day 1-3,5509052.74786199
2,2026-01-29 12:00:00,Columbia-Pacific Northwest,2026-01-30,2026-02-01,Day 1-3,705817.3134749037
3,2026-01-29 12:00:00,Missouri Basin,2026-01-30,2026-02-01,Day 1-3,535205.6023810036
4,2026-01-29 12:00:00,North Atlantic-Appalachian,2026-01-30,2026-02-01,Day 1-3,10363919.605481459
